In [ ]:
# 示例解析
## 账本应用
程序在进入时，会创建TradingEnv类的实例env：
```python
env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.SimpleOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)
```
这里我们先来研究action_scheme属性。重点关注evn.action_scheme.action_space属性。由上面的代码可以看出，env.action_scheme是iqt.env.default.actions.SimpleOrders类，action_space是其计算属性，如下所示：
```python
@property
def action_space(self) -> Space:
    if not self._action_space:
        self.actions = product(
            self.criteria,
            self.trade_sizes,
            self.durations,
            [TradeSide.BUY, TradeSide.SELL]
        )
        print('self.actions 1: {0};'.format(self.actions))
        self.actions = list(self.actions)
        print('self.actions 2: {0};'.format(self.actions))
        self.actions = list(product(self.portfolio.exchange_pairs, self.actions))
        print('self.actions 3: {0};'.format(self.actions))
        self.actions = [None] + self.actions
        print('self.actions 4: {0};'.format(self.actions))
        self._action_space = Discrete(len(self.actions))
    return self._action_space
```
运行结果如下所示：
```bash
self.actions 1: <itertools.product object at 0x000001B16C0F53C0>;
self.actions 2: 
[
    (None, 0.1, None, <TradeSide.BUY: 'buy'>), 
    (None, 0.1, None, <TradeSide.SELL: 'sell'>), 
    (None, 0.2, None, <TradeSide.BUY: 'buy'>), 
    ......
    (None, 0.9, None, <TradeSide.SELL: 'sell'>), 
    (None, 1.0, None, <TradeSide.BUY: 'buy'>), 
    (None, 1.0, None, <TradeSide.SELL: 'sell'>)
];

self.actions 3: 
[
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.BUY: 'buy'>)), 
    ......
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.SELL: 'sell'>))
];
    
self.actions 4: 
[
    None, 
    ################################################################################################
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.SELL: 'sell'>)),
    ......
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.SELL: 'sell'>))
];
action_space: Discrete(101);
```
其中action_space为gym.spaces.Descrete(101)，其取值范围为0$\sim$100，对应于step4打印出的某一行，第一项为操作标的，第二项为：
1. cretia：代表市价、限价等订单类型；
2. trade_size：操作比例，如bitstamp: USD/ETH, (None, 0.2, None, <TradeSide.BUY: 'buy'>代表用当前资金的20%买入ETH；
3. duration：None；
4. 买卖方向；

当为1$\sim$100时，代表对交易对的买卖操作。当为0时，代表什么都不做。

接下来对环境进行重置：
```python
def reset(self) -> 'np.array':
    self.episode_id = str(uuid.uuid4())
    self.clock.reset()
    print('components:')
    for c in self.components.values():
        print('    {0};'.format(c))
        if hasattr(c, "reset"):
            c.reset()
    obs = self.observer.observe(self)
    print('init obs: {0};'.format(obs))
    self.clock.increment()
    return obs
```
运行结果如下所示：
```python
components:
    <iqt.env.default.actions.SimpleOrders object at 0x000001C147A5ED00>;
    <iqt.env.default.rewards.SimpleProfit object at 0x000001C147A5EDF0>;
    <iqt.env.default.observers.iqtObserver object at 0x000001C147A5EEB0>;
    <iqt.env.default.stoppers.MaxLossStopper object at 0x000001C147A5EE80>;
    <iqt.env.default.informers.iqtInformer object at 0x000001C147A5E430>;
    <iqt.env.default.renderers.EmptyRenderer object at 0x000001C147A5ED60>;
init obs: [[2569630.2  536484.2]]
```
环境的components包括action_scheme、reward_scheme、observer、stopper（用于确定停止条件，如亏光所有资金）、info（额外信息）、render用于可视化。obs是环境当前状态，在本例中，我们只观察bitstamp交易所的BTC和LTC，其数据文件格式为：

|No.|date|unix|open|high|low|close|volume|
|-|-|-|-|-|-|-|-|
|0|2018/5/15 6:00|1526364000|147.2|148.7|147.01|147.2|1907.28|

我们的obs是bitstamp交易所ETH和LTC两个币种的交易量。

然后进入无限循环，在此无限循环的每一步，首先从action_space中随机取出一个操作，然后调用iqt.env.generic.environment.py::TradingEnv.step方法：
```python
    def step(self, action: Any) -> 'Tuple[np.array, float, bool, dict]':
        """Makes on step through the environment.
        Parameters
        ----------
        action : Any
            An action to perform on the environment.
        Returns
        -------
        `np.array`
            The observation of the environment after the action being
            performed.
        float
            The computed reward for performing the action.
        bool
            Whether or not the episode is complete.
        dict
            The information gathered after completing the step.
        """
        self.action_scheme.perform(self, action)
        obs = self.observer.observe(self)
        reward = self.reward_scheme.reward(self)
        done = self.stopper.stop(self)
        info = self.informer.info(self)
        self.clock.increment()
        return obs, reward, done, info
```
env.action_scheme是iqt.env.default.actions.SimpleOrders，其父类是iqt.env.default.actions.IqtActionScheme，会调用该类的perform方法：
```python
def perform(self, env: 'TradingEnv', action: Any) -> None:
    """Performs the action on the given environment.
    Under the TT action scheme, the subclassed action scheme is expected
    to provide a method for getting a list of orders to be submitted to
    the broker for execution in the OMS.
    Parameters
    ----------
    env : 'TradingEnv'
        The environment to perform the action on.
    action : Any
        The specific action selected from the action space.
    """
    orders = self.get_orders(action, self.portfolio)
    for order in orders:
        if order:
            logging.info('Step {}: {} {}'.format(order.step, order.side, order.quantity))
            self.broker.submit(order)
    self.broker.update()
```
我们首先来看iqt.env.default.actions.SimpleOrders.get_orders方法：
```python
def get_orders(self,
               action: int,
               portfolio: 'Portfolio') -> 'List[Order]':
    if action == 0:
        return []
    (ep, (criteria, proportion, duration, side)) = self.actions[action]
    print('trading_pair: {0}; criteria: {1}; proportion: {2}; duration: {3}; side: {4};'.format(ep, criteria, proportion, duration, side))
    instrument = side.instrument(ep.pair)
    print('instrument: {0};'.format(instrument))
    wallet = portfolio.get_wallet(ep.exchange.id, instrument=instrument)
    print('wallet: {0};'.format(wallet))
    balance = wallet.balance.as_float()
    print('balance: {0};'.format(balance))
    size = (balance * proportion)
    size = min(balance, size)
    print('size: {0};'.format(size))
    quantity = (size * instrument).quantize()
    print('quantity: {0};'.format(quantity))
    v1 = 10 ** -instrument.precision
    v2 = self.min_order_pct * portfolio.net_worth
    v3 = self.min_order_abs
    print('v1={0}; v2={1}; v3={2};'.format(v1, v2, v3))
    print('min_order_pct: {0}; net_worth: {1};'.format(self.min_order_pct, portfolio.net_worth))
    if size < 10 ** -instrument.precision \
            or size < self.min_order_pct * portfolio.net_worth \
            or size < self.min_order_abs:
        return []
    order = Order(
        step=self.clock.step,
        side=side,
        trade_type=self._trade_type,
        exchange_pair=ep,
        price=ep.price,
        quantity=quantity,
        criteria=criteria,
        end=self.clock.step + duration if duration else None,
        portfolio=portfolio
    )
    print('order_listener: {0};'.format(self._order_listener))
    if self._order_listener is not None:
        order.attach(self._order_listener)
    return [order]
```
程序运行结果如下所示：
```bash
trading_pair: bitstamp:USD/BTC; criteria: None; proportion: 1.0; duration: None; side: sell;
instrument: BTC;
wallet: <Wallet: balance=500.00000000 BTC, locked=0.00000000 BTC>;
balance: 500.0;
size: 500.0;
quantity: 500.00000000 BTC;
v1=1e-08; v2=0.0; v3=0.0;
min_order_pct: 0.0; net_worth: 98055371.0;
order_listener: None;
```
我们回到IqtActionScheme.perform方法，get_orders会生成订单，接下来我们就取出订单，依次将订单提交到券商broker，这个方法只是简单的将订单添加到未执行的订单列表中。接下来我们就调用iqt.oms.orders.order.Broker.update方法：
```python
def update(self) -> None:
    """Updates the brokers order management system.
    The broker will look through the unexecuted orders and if an order
    is ready to be executed the broker will submit it to the executed
    list and execute the order.
    Then the broker will find any orders that are active, but expired, and
    proceed to cancel them.
    """
    for order in self.unexecuted:
        if order.is_executable:
            self.unexecuted.remove(order)
            self.executed[order.id] = order
            order.attach(self)
            order.execute()
    for order in self.unexecuted + list(self.executed.values()):
        if order.is_active and order.is_expired:
            self.cancel(order)
```

首先将订单从未执行订单列表中移除，随后添加到已经执行的订单字典中。然后将broker自己加入到order的监听者中，然后调用订单的执行方法。接着将未执行订单列表和已执行订单字典中处于活跃状态但是已经过期的订单，调用订单的取消操作。

我们首先来看订单的执行操作，订单为iqt.oms.orders.order.Order.execute方法：
```python
def execute(self) -> None:
    """Executes the order."""
    self.status = OrderStatus.OPEN
    if self.portfolio.order_listener:
        self.attach(self.portfolio.order_listener)
    for listener in self.listeners or []:
        listener.on_execute(self)
    self.exchange_pair.exchange.execute_order(self, self.portfolio)
```
